In [5]:
import pandas as pd

In [6]:
df_train=pd.read_csv("data/train.csv")
df_test=pd.read_csv("data/test.csv")

In [7]:
df_train

,pid,text,labels
0,train_pid_7991,At this point just genuinely curious what sort...,0
1,train_pid_7992,I have literally tried everything.... : I'm st...,0
2,train_pid_7995,I'm really struggling : So I don't know how to...,0
3,train_pid_7996,My meds aren’t making my depression any better...,0
4,train_pid_7997,"Hi I'm unwell : I'm 21 now, ""vice ridden"", and...",0
...,...,...,...
6001,dev_pid_3724,What's even the point ? : [removed],2
6002,dev_pid_3725,I just want everyone to know that their not al...,2
6003,dev_pid_3726,I hope I get the coronavirus : I hope I get th...,2
6004,dev_pid_3727,Sometimes I really wonder if being less ugly w...,2


In [8]:
df_train['labels'].value_counts()

labels
1    3101
2    2255
0     650
Name: count, dtype: int64

In [9]:
df_train

,pid,text,labels
0,train_pid_7991,At this point just genuinely curious what sort...,0
1,train_pid_7992,I have literally tried everything.... : I'm st...,0
2,train_pid_7995,I'm really struggling : So I don't know how to...,0
3,train_pid_7996,My meds aren’t making my depression any better...,0
4,train_pid_7997,"Hi I'm unwell : I'm 21 now, ""vice ridden"", and...",0
...,...,...,...
6001,dev_pid_3724,What's even the point ? : [removed],2
6002,dev_pid_3725,I just want everyone to know that their not al...,2
6003,dev_pid_3726,I hope I get the coronavirus : I hope I get th...,2
6004,dev_pid_3727,Sometimes I really wonder if being less ugly w...,2


In [11]:
df_train.drop(['pid'], axis=1, inplace=True)

In [12]:
df_train

,text,labels
0,At this point just genuinely curious what sort...,0
1,I have literally tried everything.... : I'm st...,0
2,I'm really struggling : So I don't know how to...,0
3,My meds aren’t making my depression any better...,0
4,"Hi I'm unwell : I'm 21 now, ""vice ridden"", and...",0
...,...,...
6001,What's even the point ? : [removed],2
6002,I just want everyone to know that their not al...,2
6003,I hope I get the coronavirus : I hope I get th...,2
6004,Sometimes I really wonder if being less ugly w...,2


In [13]:
df_test

,pid,text,labels
0,test_pid_1,Im scared : This is it. I lie to myself every ...,1
1,test_pid_2,New to this but just wanted to vent : I just f...,1
2,test_pid_3,I’m sad : It’s kinda always been an issue. I w...,1
3,test_pid_4,Lonely but not alone. : All of my immediately ...,1
4,test_pid_5,This year has been trash. : I dont know why I’...,1
...,...,...,...
3240,test_pid_3241,"Feeling lonely. : Hi reddit, I haven’t posted ...",0
3241,test_pid_3242,When would suicide be right? : So I got back f...,0
3242,test_pid_3243,Lowest I’ve ever been ever. : To make a long s...,0
3243,test_pid_3244,Does the Toxoplasma Gondii ruined my life ? (f...,0


In [16]:
df_test.drop('pid', axis=1,inplace=True)

In [17]:
from transformers import BertTokenizer

In [18]:
import torch
from torch.utils.data import TensorDataset

In [19]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
def encode(text,label):
    encodings=tokenizer(text.tolist(),padding='max_length',truncation=True,max_length=128,return_tensors='pt')
    label=torch.tensor(label.tolist())
    return TensorDataset(encodings["input_ids"],encodings["attention_mask"],label)

In [21]:
train_dataset=encode(df_train['text'],df_train['labels'])
test_dataset=encode(df_test['text'],df_test['labels'])

In [22]:
from torch.utils.data import DataLoader
from torchsampler import ImbalancedDatasetSampler

In [23]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

In [24]:
from transformers import BertForSequenceClassification 
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
def get_label(dataset):
    # Access labels directly from dataset.tensors
    return dataset.tensors[2]

In [26]:
train_loader=DataLoader(train_dataset,batch_size=16,shuffle=True)
optimizer=AdamW(model.parameters(),lr=2e-5, eps=1e-8)

In [27]:
epochs=3
train_steps=len(train_loader)*epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    num_training_steps=train_steps
)

In [28]:
import torch.nn.functional as F

def focal_loss(outputs, targets, alpha=None, gamma=2):
    """
    outputs: model logits (batch_size, num_classes)
    targets: true labels (batch_size)
    alpha: class weights tensor (for imbalance) or None
    gamma: focusing parameter (default=2)
    """
    ce_loss = F.cross_entropy(outputs, targets, weight=alpha, reduction='none')
    pt = torch.exp(-ce_loss)   # probability of the true class
    focal = ((1 - pt) ** gamma * ce_loss).mean()
    return focal

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

alpha = torch.tensor([1.0, 0.8, 0.6]).to(device)

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        b_input_ids, b_attention_mask, b_labels = [t.to(device) for t in batch]

        optimizer.zero_grad()
        outputs = model(
            input_ids=b_input_ids,
            attention_mask=b_attention_mask
        )
        logits = outputs.logits

        loss = focal_loss(logits, b_labels, alpha=alpha, gamma=0.5)
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    print(f"Epoch {epoch+1} done")


Epoch 1 done
Epoch 2 done
Epoch 3 done


In [30]:
test_loader=DataLoader(test_dataset,batch_size=16, shuffle=True)

In [31]:
from sklearn.metrics import classification_report

model.eval()
preds, true_labels = [], []
for batch in test_loader:
    with torch.no_grad():
        outputs = model(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
    logits = outputs.logits
    preds.extend(torch.argmax(logits, dim=1).cpu().numpy())
    true_labels.extend(batch[2].numpy())

print(classification_report(true_labels, preds))


              precision    recall  f1-score   support

           0       0.39      0.57      0.46       228
           1       0.75      0.74      0.74      2169
           2       0.48      0.43      0.46       848

    accuracy                           0.65      3245
   macro avg       0.54      0.58      0.55      3245
weighted avg       0.65      0.65      0.65      3245

